In [16]:
#Resources: https://splinter.readthedocs.io/en/latest/#features 
#http://web.stanford.edu/~zlotnick/TextAsData/Web_Scraping_with_Beautiful_Soup.html
#https://www.crummy.com/software/BeautifulSoup/bs4/doc/

#This program conducts a batch BLAST of multiple genes from JGI and stores the top ten results for each BLAST.

In [1]:
#Section 1: imports the necessary libraries and creates the arrays needed to store information from JGI and BLAST (RUN THIS)
#You have to make sure that you have the splinter and BeautifulSoup libraries (reference resources above for download process)
from splinter import Browser
from bs4 import BeautifulSoup
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
import numpy as np
from collections import Counter

In [2]:
# Create scaffold spreadsheet with Filtered Community Composition
# Make data frames for every sheet
xls = pd.ExcelFile('C:/Users/coverney/Desktop/Research/Meta/Data/Filtered community comparison.xlsx')
sample_names = ['FCR15b', 'FCFTC3', 'SCH14c', 'SCB14c', 'FCX16c', 
                'FCS17a', 'FCW18b', 'SCQ17a', 'SCR11c', 'FCP16c', 'SCD17c']
dfs = []
for index, name in enumerate(sample_names):
    temp_df = pd.read_excel(xls, sample_names[index])
    dfs.append(temp_df)

In [3]:
# Create DataFrames
headers = dfs[0].loc[0, 'Scaffold ID':]
scaffold = pd.DataFrame(columns=headers.index)
# references 
references = pd.DataFrame(columns=['sample', 'scaffold id', 'gene #', 'pfam name'])

In [23]:
temp_df = dfs[0]
temp_pfams = temp_df['GH1']

In [4]:
# Go through Lineage Phylum column and if there is a nan
# and gene count is >= 5, then add info to scaffold and references spreadsheet
for index, df in enumerate(dfs[:1]):
#for index, df in enumerate(dfs):
    current_pfam = 'GH1'
    temp_pfams = df['GH1']
    temp_genecounts = df['Gene Count']
    temp_phyla = df['Lineage Phylum']
    for index2, pfam in enumerate(temp_pfams):
        if pfam is not np.nan:
            current_pfam = pfam
        if temp_phyla[index2] is np.nan:
            # Check gene count
            temp_genecount = temp_genecounts[index2]
            if (temp_genecount is not np.nan) and (temp_genecount >= 5):
                temp_row = df.loc[index2, 'Scaffold ID':]
                scaffold = scaffold.append(temp_row, ignore_index=True)
                temp_row2 = {'sample':sample_names[index], 'scaffold id':temp_row[0], 'gene #':index2, 'pfam name':current_pfam}
                references = references.append(temp_row2, ignore_index=True)

In [68]:
scaffold.to_excel("C:/Users/coverney/Desktop/Research/Meta/Data/scaffolds.xlsx", index=False)

In [112]:
#Section 2: opens up a Chrome browser (RUN THIS)
executable_path = {'executable_path':'C:\Program Files\chromedriver.exe'}
browser = Browser('chrome', **executable_path)

In [119]:
scaffold_cart_url = "https://img.jgi.doe.gov/cgi-bin/mer/main.cgi?section=ScaffoldCart&page=index"
browser.visit(scaffold_cart_url)

In [89]:
len(scaffold['Scaffold ID'].unique())

32

In [120]:
# Go through scaffolds in scaffold cart and store urls to gene counts
gene_count_urls = []
scaffold_ids = []
gene_counts = []
for i in range(0,len(scaffold['Scaffold ID'].unique())):
    button = browser.find_by_id('yui-rec' + str(i))
    soup = BeautifulSoup(button.html, "html.parser")
    links = soup.find_all('a')
    gene_count_urls.append(links[2].get('href'))
    gene_counts.append(links[2].get_text())
    scaffold_ids.append(links[0].get_text())

In [142]:
#Section 3: obtains the top three amino acid sequences from the gene list and stores them in the sequences array
#The gene_url is the link to the gene list of a particular scaffold (replace this url with your own), you may be prompted to sign into JGI. 
#If so, sign in and then re-run this section. 
#Change the number of genes to blast by changing the "range(0,3)" section to "range(0,n)", n being the number you want
import time
prefix = "https://img.jgi.doe.gov/cgi-bin/mer/"
#gene_sequences = pd.DataFrame(columns=['scaffold id', 'num_seqs', 'seq1', 'seq2', 'seq3', 'seq4', 'seq5'])
#for index, url in enumerate(gene_count_urls[:1]):
for index, url in enumerate(gene_count_urls[29:]):
    temp_sequences = []
    count = 0
    ind = 0
    gene_count = int(gene_counts[index+29])
    while(count < 5 and ind < gene_count):
        browser.visit(prefix+url)
        button = browser.find_by_id('yui-rec' + str(ind))
        soup = BeautifulSoup(button.html, "html.parser")
        # Make sure Locus Type is CDS
        links = soup.find_all('a')
        geneid = links[0].get_text()
        if soup.get_text()[len(geneid): len(geneid)+3] == 'CDS':
            gene_name = soup.get_text()[len(geneid)+3+len(geneid):]
            if 'hypothetical protein' not in gene_name:
                gene_url2 = links[0].get('href')
                browser.visit(prefix + gene_url2)
                soup2 = BeautifulSoup(browser.html, "html.parser")
                tab = soup2.table
                tabs = tab.find_all('tr')
                gene_url3 = tabs[15].a.get('href')
                browser.visit(prefix + gene_url3)
                soup3 = BeautifulSoup(browser.html, "html.parser")
                a = soup3.pre.get_text()
                b = soup3.pre.font.get_text()
                amino = "".join(a.rsplit(b))
                amino = amino.replace('\n', '')
                temp_sequences.append(amino)
                count += 1
            else:
                print('hypo protein')
        ind += 1
    new_row = {'scaffold id':scaffold_ids[index], 'num_seqs':count}
    for i in range(0, count):
        temp_name = 'seq' + str(i+1)
        new_row.update({temp_name:temp_sequences[i]})
    gene_sequences = gene_sequences.append(new_row, ignore_index=True)

gene_sequences.head()

hypo protein
hypo protein
hypo protein
hypo protein
hypo protein
hypo protein
hypo protein
hypo protein
hypo protein
hypo protein
hypo protein
hypo protein
hypo protein
hypo protein
hypo protein
hypo protein
hypo protein
hypo protein


,scaffold id,num_seqs,seq1,seq2,seq3,seq4,seq5
0,Ga0136444_100052,5,MATDELLRPTVRLDVFDGPLDLLLYLVHSHELDPRTIPVSQIADQY...,MLLPLPVDALRPAFLELRARHRPVVVTSPTGSGKSTRVPLWCAETG...,MLASFLERQSDLAGLLGAGAWLSLLWVRRAAALPGGKGFDAAAVAL...,MNGTGPDETVWTALARCWEELFPLRPARTAMCRALVSAGGRLLDAG...,MVEEDSPDFTSFYTTTAVSERSLSFMGTAIGRDFGREDLEDRANGA...
1,Ga0136444_100059,5,MRIRLVCFGKLVPKPLGPAVEEYAKRLEKLCRLEIHELPEAQGPDA...,MSEPLFVQPFQVRDHECDLQGIVNNAVYQNYLEHARHEYLRSRGVD...,MGDGSNVRKRAGFTMVEVITVVVVVGLLAGLVVPRFFKAQGRRQLE...,VDLGRQLEEIVNRRARRAYTLVEVLVAMVILGLVLPGLATMLVSSR...,MTRRGLTLLELLVAMILTGILAWLALGMFTGESANYRRTREKVKMQ...
2,Ga0136444_100076,5,MHVVLFCALSLDGCLAREDGSLDWLPQDPGATDDMGFSALLASVDG...,MSSPSHPVGPAIALRHDGDFEGFLTAVFEIFRLRLESAVPVARRLH...,VQSKLSILSAAAKYDVSCSSSGSKRATPSDGLGNGAPAGICHSFTD...,MIRIAHTQLSDAREAVKELRSQLDPIQPRLLLCFSSTRQDPHEMAR...,MAYRIATLNKISPEGLNLFGSEYEVADGLSSPDAILVRSAQLDTDG...
3,Ga0136444_100106,5,MRTGLLLAVVALVVLAGFAGARALRSQGGGAVPKEATVRDFDEHGE...,LKRTATLSVLALAAAAAAAQTIRVHADAPGHRISPAIYGINSLATG...,MKIAAVFTGGTIGSLATTESVQLDADPPRRVLESWLGSHPGTDVEF...,MSAGTFEIHARCGKARTGVLTTDHGDIRTPAFMPVGTAGSVKSLTS...,VPRYNPSSIEPKWQSVWAANKTFRTPGIAELDRSRPKFYALDMFPY...
4,Ga0136444_100122,5,PPGAIVPQNGRNPFSSRRWLDAQDRLLPLDSLIPWPGGASIVLRKL...,MDIAFLGLGIMGSRMAANLAAAGHTVRVWNRSPEKALALASHGCRP...,MYTIANLATPRSFIDEFFREVERASSATPAFQPAADVVEVEGGWKV...,MTRPVGLLLANLGTPSAPTAAAVRPYLRQFLTDRRVIDIPWLPRQL...,MAPATDSDGRPTVDGRSHPSFDPPYPRKSTMNPQRFLPLLAATTLL...


In [151]:
gene_sequences.shape[0]

32

In [144]:
gene_sequences.to_excel("C:/Users/coverney/Desktop/Research/Meta/Data/gene_sequences1.xlsx", index=False)

In [3]:
# read in gene sequences
gene_sequences = pd.read_excel('C:/Users/coverney/Desktop/Research/Meta/Data/gene_sequences1.xlsx')
gene_sequences.head()

,scaffold id,num_seqs,seq1,seq2,seq3,seq4,seq5
0,Ga0136444_100052,5,MATDELLRPTVRLDVFDGPLDLLLYLVHSHELDPRTIPVSQIADQY...,MLLPLPVDALRPAFLELRARHRPVVVTSPTGSGKSTRVPLWCAETG...,MLASFLERQSDLAGLLGAGAWLSLLWVRRAAALPGGKGFDAAAVAL...,MNGTGPDETVWTALARCWEELFPLRPARTAMCRALVSAGGRLLDAG...,MVEEDSPDFTSFYTTTAVSERSLSFMGTAIGRDFGREDLEDRANGA...
1,Ga0136444_100059,5,MRIRLVCFGKLVPKPLGPAVEEYAKRLEKLCRLEIHELPEAQGPDA...,MSEPLFVQPFQVRDHECDLQGIVNNAVYQNYLEHARHEYLRSRGVD...,MGDGSNVRKRAGFTMVEVITVVVVVGLLAGLVVPRFFKAQGRRQLE...,VDLGRQLEEIVNRRARRAYTLVEVLVAMVILGLVLPGLATMLVSSR...,MTRRGLTLLELLVAMILTGILAWLALGMFTGESANYRRTREKVKMQ...
2,Ga0136444_100076,5,MHVVLFCALSLDGCLAREDGSLDWLPQDPGATDDMGFSALLASVDG...,MSSPSHPVGPAIALRHDGDFEGFLTAVFEIFRLRLESAVPVARRLH...,VQSKLSILSAAAKYDVSCSSSGSKRATPSDGLGNGAPAGICHSFTD...,MIRIAHTQLSDAREAVKELRSQLDPIQPRLLLCFSSTRQDPHEMAR...,MAYRIATLNKISPEGLNLFGSEYEVADGLSSPDAILVRSAQLDTDG...
3,Ga0136444_100106,5,MRTGLLLAVVALVVLAGFAGARALRSQGGGAVPKEATVRDFDEHGE...,LKRTATLSVLALAAAAAAAQTIRVHADAPGHRISPAIYGINSLATG...,MKIAAVFTGGTIGSLATTESVQLDADPPRRVLESWLGSHPGTDVEF...,MSAGTFEIHARCGKARTGVLTTDHGDIRTPAFMPVGTAGSVKSLTS...,VPRYNPSSIEPKWQSVWAANKTFRTPGIAELDRSRPKFYALDMFPY...
4,Ga0136444_100122,5,PPGAIVPQNGRNPFSSRRWLDAQDRLLPLDSLIPWPGGASIVLRKL...,MDIAFLGLGIMGSRMAANLAAAGHTVRVWNRSPEKALALASHGCRP...,MYTIANLATPRSFIDEFFREVERASSATPAFQPAADVVEVEGGWKV...,MTRPVGLLLANLGTPSAPTAAAVRPYLRQFLTDRRVIDIPWLPRQL...,MAPATDSDGRPTVDGRSHPSFDPPYPRKSTMNPQRFLPLLAATTLL...


In [4]:
gene_sequences_seqs = gene_sequences.loc[:, 'seq1':]
gene_sequences_seqs.iloc[12].values[0]

'MRIAYQIEGSRLAQCDAEQAKVLVYVAPTEADRRELVDLHGLDEHTLNSALDPDELPRLEFEADHTALIMKRPRPHSPRDKYLFKVASFGLFLFKDKMILVAGDEVQFFEGKLFQRITGVADLALRVVYRSIWHFVDHLKVINMISDSLEEKINASMENRYLIDLFTLEKSLVYYLNAISGNGVVFERLKLNALRVGFSIDMTEMLDDLVIENNQCFKQAEIYSNILAGLMDARASIVSNNLNMLMKNLNVITIAIMVPTFVVSAFSMNVILPWGMAEAKWSFWFVMGLATISVFGYMWRSRKW*'

In [2]:
executable_path = {'executable_path':'C:\Program Files\chromedriver.exe'}
browser = Browser('chrome', **executable_path)

In [5]:
def wait(browser):
    found_title = browser.find_by_text('                            \n                   BLAST Results\n                ')
    if found_title:
        return 1
    else:
        return 0

In [18]:
def blast(seq, browser):
    blast_url = "https://blast.ncbi.nlm.nih.gov/Blast.cgi?PAGE=Proteins"
    browser.visit(blast_url)
    browser.fill('QUERY', seq)
    button = browser.find_by_id('b1')
    button.click()
    move_on = wait(browser)
    while not move_on:
        move_on = wait(browser)
    table = browser.find_by_id('dscTable')
    result = []
    if len(table) != 0:
        for i in range(1,11):  
            try:
                res = table.find_by_id('deflnDesc_' + str(i)).html
                pos = res.find('[')
                result.append(res[pos+1:-1])
            except:
                break
    return result

In [7]:
def find_phylum(val, browser):
    tax_website = 'https://www.ncbi.nlm.nih.gov/taxonomy'
    browser.visit(tax_website)
    browser.fill('term', val)
    button = browser.find_by_id('search')
    button.click()
    wait = browser.find_by_id('messagearea')
    while len(wait) is 0:
        wait = browser.find_by_id('messagearea')
    search_area = browser.find_by_id('maincontent')
    soup = BeautifulSoup(search_area.html, "html.parser")
    links = soup.find_all('a')
    for link in links:
        if link.get_text() == val:
            url = 'https://www.ncbi.nlm.nih.gov/'+link.get('href')
            break
    browser.visit(url)
    soup = BeautifulSoup(browser.html, "html.parser")
    try:
        phylum = soup.find(title='phylum').get_text()
        return phylum
    except:
        return 'none'

In [17]:
def find_phylum_of_gene(phyla):
    counter = Counter(phyla).most_common(1)
    if counter[0][1] >= 3:
        phylum = counter[0][0]
    else:
        phylum = 'none'
    return phylum

In [9]:
def find_phylum_of_scaffold(phyla):
    counter = Counter(phyla).most_common(1)
    if counter[0][0] != 'none' and counter[0][1] >= 2:
        phylum = counter[0][0]
    else:
        phylum = 'Unclassified'
    return phylum

In [10]:
def find_phyla_of_sample(gene_sequences_seqs, rng):
    overall_phyla = []
    # Go through every scaffold
    for index in rng:
        scaffold_phyla = []
        # Get the gene sequences of the scaffold
        sequences = gene_sequences_seqs.iloc[index].values[1:]
        # Go through the sequences and BLAST
        index2 = 0
        three_phyla = 0
        while index2 < len(sequences) and three_phyla < 3:
            seq = sequences[index2]
            blast_results = blast(seq, browser) # results for one gene
            if not blast_results:
                gene_phyla = []
                for elm in blast_results:
                    gene_phyla.append(find_phylum(elm, browser))
                # Find phylum of gene 
                gene_phylum = find_phylum_of_gene(gene_phyla)
                print(gene_phylum)
                # If phyla is not none then increment three_phyla
                if gene_phylum != 'none':
                    three_phyla += 1
                    scaffold_phyla.append(gene_phylum)
            index2 += 1
        # Find phylum of scaffold
        scaffold_phylum = find_phylum_of_scaffold(scaffold_phyla)
        overall_phyla.append(scaffold_phylum)
    return overall_phyla

In [13]:
#overall_phyla = find_phyla_of_sample(gene_sequences_seqs, range(0,gene_sequences_seqs.shape[0]))
overall_phyla = find_phyla_of_sample(gene_sequences_seqs, range(3,4))
print(overall_phyla)
#gene_sequences['phylum'] = pd.Series(overall_phyla)
#gene_sequences.to_excel("C:/Users/coverney/Desktop/Research/Meta/Data/gene_sequences2.xlsx", index=False)

Verrucomicrobia


IndexError: list index out of range

In [ ]:
# Reference NCBI Taxonomy website
tax_website = 'https://www.ncbi.nlm.nih.gov/taxonomy'
all_phylas = []
overall_phyla = []
# go through scaffolds
#for results in gene_sequences['results']:
for results in all_results:
    phylas = []
    # go through genes
    for res in results:
        # go through results from one gene
        phyla = []
        for val in res:
            browser.visit(tax_website)
            browser.fill('term', val)
            button = browser.find_by_id('search')
            button.click()
            wait = browser.find_by_id('messagearea')
            while len(wait) is 0:
                wait = browser.find_by_id('messagearea')
            search_area = browser.find_by_id('maincontent')
            soup = BeautifulSoup(search_area.html, "html.parser")
            links = soup.find_all('a')
            for link in links:
                if link.get_text() == val:
                    url = 'https://www.ncbi.nlm.nih.gov/'+link.get('href')
                    break
            browser.visit(url)
            soup = BeautifulSoup(browser.html, "html.parser")
            try:
                phylum = soup.find(title='phylum').get_text()
                phyla.append(phylum)
            except:
                continue
        gene_phylum_counter = Counter(phyla).most_common(1)
        if gene_phylum_counter[0][1] >= 3:
            gene_phylum = gene_phylum_counter[0][0]
        else:
            gene_phylum = 'none'
        phylas.append(gene_phylum)
    top_phylum = Counter(phylas).most_common(1)
    if top_phylum[0][0] != 'none' and top_phylum[0][1] >= 3:
        overall_phylum = top_phylum[0][0]
    else:
        overall_phylum = 'Unclassified'
    overall_phyla.append(overall_phylum)
    all_phylas.append(phylas)
print(overall_phyla)
# gene_sequences['phyla'] = pd.Series(overall_phyla)
# gene_sequences['phylas'] = pd.Series(all_phylas)

In [156]:
#Section 4: takes the sequences from section 3 and BLASTs them.
blast_url = "https://blast.ncbi.nlm.nih.gov/Blast.cgi?PAGE=Proteins"
all_results = []
gene_sequences_seqs = gene_sequences.loc[:, 'seq1':]
for index in range(gene_sequences_seqs.shape[0]):
# for index in range(0,1):
    results = []
    row = gene_sequences_seqs.iloc[index].values
    for seq in row:
        browser.visit(blast_url)
        browser.fill('QUERY', seq)
        button = browser.find_by_id('b1')
        button.click()
        wait = browser.find_by_id('summarylists')
        while not wait:
            wait = browser.find_by_id('summarylists')
        table = browser.find_by_id('dscTable')
        if len(table) != 0:
            result = []
            for i in range(1,6):  
                try:
                    res = table.find_by_id('deflnDesc_' + str(i)).html
                    pos = res.find('[')
                    result.append(res[pos+1:-1])
                except:
                    break
            results.append(result)
    all_results.append(results)
#gene_sequences['results'] = pd.Series(all_results)
print(all_results)

[[['Candidatus Edwardsbacteria bacterium RifOxyC12_full_54_24', 'Desulfobacteraceae bacterium', 'Deltaproteobacteria bacterium HGW-Deltaproteobacteria-15', 'candidate division Zixibacteria bacterium CG_4_9_14_3_um_filter_46_8', 'Ignavibacteria bacterium RBG_13_36_8'], ['Fibrobacter sp. UWEL', 'Fibrobacter sp. UWH8', 'Fibrobacter sp. UWH8', 'Fibrobacter sp. UWOV1', 'Fibrobacter sp. UWCM'], ['Spirochaetes bacterium', 'Spirochaetae bacterium HGW-Spirochaetae-1', 'Turneriella parva', 'Acidobacteria bacterium', 'Thermococcus profundus'], ['Fibrobacter sp. UWB1', 'Fibrobacter sp. UWOV1', 'Fibrobacter', 'Fibrobacter sp. UWB1', 'Fibrobacter sp. UWB8'], ['Fibrobacter sp. UWR2', 'Geminisphaera colitermitum', 'Fibrobacter', 'Fibrobacter sp. UWP2', 'Fibrobacter sp. UWEL']]]


In [114]:
gene_sequences.to_excel("C:/Users/coverney/Desktop/Research/Meta/Data/gene_sequences2.xlsx", index=False)

In [160]:
from collections import Counter
# Reference NCBI Taxonomy website
tax_website = 'https://www.ncbi.nlm.nih.gov/taxonomy'
all_phylas = []
overall_phyla = []
# go through scaffolds
#for results in gene_sequences['results']:
for results in all_results:
    phylas = []
    # go through genes
    for res in results:
        # go through results from one gene
        phyla = []
        for val in res:
            browser.visit(tax_website)
            browser.fill('term', val)
            button = browser.find_by_id('search')
            button.click()
            wait = browser.find_by_id('messagearea')
            while len(wait) is 0:
                wait = browser.find_by_id('messagearea')
            search_area = browser.find_by_id('maincontent')
            soup = BeautifulSoup(search_area.html, "html.parser")
            links = soup.find_all('a')
            for link in links:
                if link.get_text() == val:
                    url = 'https://www.ncbi.nlm.nih.gov/'+link.get('href')
                    break
            browser.visit(url)
            soup = BeautifulSoup(browser.html, "html.parser")
            try:
                phylum = soup.find(title='phylum').get_text()
                phyla.append(phylum)
            except:
                continue
        gene_phylum_counter = Counter(phyla).most_common(1)
        if gene_phylum_counter[0][1] >= 3:
            gene_phylum = gene_phylum_counter[0][0]
        else:
            gene_phylum = 'none'
        phylas.append(gene_phylum)
    top_phylum = Counter(phylas).most_common(1)
    if top_phylum[0][0] != 'none' and top_phylum[0][1] >= 3:
        overall_phylum = top_phylum[0][0]
    else:
        overall_phylum = 'Unclassified'
    overall_phyla.append(overall_phylum)
    all_phylas.append(phylas)
print(overall_phyla)
# gene_sequences['phyla'] = pd.Series(overall_phyla)
# gene_sequences['phylas'] = pd.Series(all_phylas)

['Fibrobacteres']


In [161]:
print(all_phylas)

[['none', 'Fibrobacteres', 'Spirochaetes', 'Fibrobacteres', 'Fibrobacteres']]


In [117]:
gene_sequences.head()

,scaffold id,seqs,phylas,results,phyla
0,Ga0136444_100052,[MATDELLRPTVRLDVFDGPLDLLLYLVHSHELDPRTIPVSQIADQ...,"[Fibrobacteres, none, none]","[[Fibrobacter sp. UWS2, Fibrobacter sp. UWS3, ...",Unclassified
1,Ga0136444_100059,[MRIRLVCFGKLVPKPLGPAVEEYAKRLEKLCRLEIHELPEAQGPD...,"[Firmicutes, none, none]","[[Clostridium, Clostridium botulinum, Clostrid...",Unclassified
2,Ga0136444_100076,[MKAVVVLALASALVGLGGCATIKVHDAVQEIGLLEPGHLEWWDDS...,"[none, none]","[[Pseudomonas, Candidatus Marinimicrobia bacte...",Unclassified
3,Ga0136444_100106,[MRTGLLLAVVALVVLAGFAGARALRSQGGGAVPKEATVRDFDEHG...,"[Verrucomicrobia, Bacteroidetes, none]","[[Verrucomicrobia bacterium LW23, Methylacidip...",Unclassified
4,Ga0136444_100122,[PPGAIVPQNGRNPFSSRRWLDAQDRLLPLDSLIPWPGGASIVLRK...,"[none, none]","[[Lentzea albida, Lentzea waywayandensis, Delt...",Unclassified


In [118]:
gene_sequences.to_excel("C:/Users/coverney/Desktop/Research/Meta/Data/gene_sequences3.xlsx", index=False)

In [42]:
#get the actual data
result = []
table = browser.find_by_id('dscTable')
for i in range(1,6):  
    res = table.find_by_id('deflnDesc_' + str(i)).html
    result.append(res)
result

['hypothetical protein [Fibrobacter sp. UWS2]',
 'hypothetical protein [Fibrobacter sp. UWS3]',
 'chromosome segregation protein ScpA [Fibrobacter sp. UWB12]',
 'condensin subunit ScpA [Fibrobacter sp. UWT3]',
 'hypothetical protein [Fibrobacter sp. UWOV1]']

In [ ]:
gene_sequences.to_excel("C:/Users/coverney/Desktop/Research/Meta/Data/gene_sequences2.xlsx", index=False)

In [ ]:

gene_sequences['phylas'] = pd.Series(phylas)
for phyla in gene_sequences['phylas']:
    counter = Counter(phyla)
    

In [18]:
# get the taxonomy report
        soup = BeautifulSoup(browser.html, "html.parser")
        links = soup.find_all('a')
        for link in links:
            if link.get_text() == '[Taxonomy reports]':
                tax_url = link.get('href')

        browser.visit('https://blast.ncbi.nlm.nih.gov/'+tax_url)
        # get phyla
        table = browser.find_by_id('lngTable')
        soup = BeautifulSoup(table.html, "html.parser")
        links = soup.find_all('a')
        phylum = links[0].get_text()
        if phylum != 'cellular organisms' and phylum != 'Bacteria':
            phyla.append(phylum)
        else:
            phyla.append('none')